## Data Importing and Cleaning

In [1]:
# Import the libraries
import pandas as pd

In [2]:
# Read in the dataset
df = pd.read_sas('../data/LLCP2017.XPT', encoding='utf-8')

/opt/conda/lib/python3.7/site-packages/pandas/io/sas/sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


In [5]:
# check number of rows and columns
df.shape

(450016, 358)

In [6]:
# check the dataframe
df.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_PA300R2,_PA30021,_PASTRNG,_PAREC1,_PASTAE1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,01302017,01,30,2017,1100.0,2017000001,2.017000e+09,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,01122017,01,12,2017,1100.0,2017000002,2.017000e+09,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,01102017,01,10,2017,1100.0,2017000003,2.017000e+09,1.0,...,3.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0,1.0,2.0
3,1.0,1.0,02082017,02,08,2017,1200.0,2017000004,2.017000e+09,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN
4,1.0,1.0,01302017,01,30,2017,1100.0,2017000005,2.017000e+09,1.0,...,3.0,2.0,2.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0


In [7]:
# select columns for analysis after screening through the codebook
df_sel = df[['DIABETE3','_BMI5', 'PHYSHLTH','MENTHLTH', '_MICHD', '_RFCHOL1', '_RFHYPE5', 'PA1VIGM_', '_RFDRHV5', '_RFSMOK3', '_INCOMG', '_EDUCAG', 'SEX', '_AGE65YR']]

In [8]:
df_sel.head()

,DIABETE3,_BMI5,PHYSHLTH,MENTHLTH,_MICHD,_RFCHOL1,_RFHYPE5,PA1VIGM_,_RFDRHV5,_RFSMOK3,_INCOMG,_EDUCAG,SEX,_AGE65YR
0,1.0,2696.0,88.0,88.0,2.0,2.0,2.0,1.800000e+02,1.0,1.0,4.0,4.0,2.0,2.0
1,3.0,2943.0,88.0,88.0,2.0,1.0,2.0,5.397605e-79,1.0,1.0,5.0,4.0,1.0,2.0
2,3.0,2504.0,88.0,88.0,2.0,2.0,1.0,NaN,1.0,1.0,9.0,1.0,1.0,2.0
3,1.0,2663.0,88.0,88.0,2.0,2.0,2.0,NaN,1.0,1.0,1.0,1.0,2.0,2.0
4,3.0,2330.0,14.0,88.0,2.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,2.0,2.0


In [9]:
# drop missing values
df_sel = df_sel.dropna()
df_sel.count()

DIABETE3    248488
_BMI5       248488
PHYSHLTH    248488
MENTHLTH    248488
_MICHD      248488
_RFCHOL1    248488
_RFHYPE5    248488
PA1VIGM_    248488
_RFDRHV5    248488
_RFSMOK3    248488
_INCOMG     248488
_EDUCAG     248488
SEX         248488
_AGE65YR    248488
dtype: int64

#### After dropping all the rows contains NA or Blanks, we got 239324 samples in the data frame.

In [10]:
## DIABETE3 - check,replace,remove
 # remove 7(don't know) and 9(refused)
 # change order and scale
  # 1 >> 1 (Yes diabetes)
  # 2 >> 1 (Yes Pre- or borderline diabetes)
  # 3 >> 0 (No)
  # 4 >> 1 (Yes Pre- or borderline diabetes)
df_sel['DIABETE3'].value_counts()
df_sel = df_sel[df_sel['DIABETE3'] != 7]
df_sel = df_sel[df_sel['DIABETE3'] != 9]
df_sel['DIABETE3'] = df_sel['DIABETE3'].astype(int)
df_sel['DIABETE3'].replace({1:1,2:1,3:0,4:1},inplace=True)

In [11]:
df_sel['DIABETE3'].value_counts()

0    212672
1     35629
Name: DIABETE3, dtype: int64

In [12]:
##  _BMI5 - check,replace,remove
 # THe original value is BMI * 100. Therefore, divide every value with 100
df_sel['_BMI5'] = df_sel['_BMI5'].div(100)
df_sel['_BMI5'].value_counts()

26.63    3123
27.46    2330
24.41    2260
27.44    2154
27.12    1938
         ... 
40.36       1
53.91       1
33.56       1
45.25       1
58.54       1
Name: _BMI5, Length: 3183, dtype: int64

In [13]:
## PHYSHLTH - check,replace,remove
 # change 88 to 0 (no bad physical health days)
 # remove 77(don't know) and 99(refused)
df_sel = df_sel[df_sel['PHYSHLTH'] != 77]
df_sel = df_sel[df_sel['PHYSHLTH'] != 99]
df_sel['PHYSHLTH'] = df_sel['PHYSHLTH'].replace({88:0})    
df_sel['PHYSHLTH'].value_counts()

0.0     161929
2.0      15167
30.0     12906
1.0      11466
3.0       8438
5.0       7217
10.0      4942
7.0       4373
4.0       4329
15.0      4042
20.0      2501
14.0      2375
6.0       1191
25.0       945
8.0        725
12.0       524
21.0       471
28.0       329
9.0        158
29.0       157
18.0       149
16.0       104
27.0        84
13.0        78
17.0        76
11.0        64
22.0        62
26.0        54
24.0        49
23.0        33
19.0        17
Name: PHYSHLTH, dtype: int64

In [14]:
## MENTHLTH - check,replace,remove
 # change 88 to 0 (no bad mental health days)
 # remove 77(don't know) and 99(refused)
df_sel = df_sel[df_sel['MENTHLTH'] != 77]
df_sel = df_sel[df_sel['MENTHLTH'] != 99]
df_sel['MENTHLTH'] = df_sel['MENTHLTH'].replace({88:0})    
df_sel['MENTHLTH'].value_counts()

0.0     169123
2.0      12873
30.0      9589
1.0       8725
5.0       8627
3.0       7370
10.0      6005
15.0      5144
4.0       3615
7.0       3271
20.0      3025
14.0      1123
6.0       1039
25.0       986
8.0        634
12.0       413
28.0       281
21.0       181
29.0       146
18.0       113
16.0        95
9.0         94
27.0        79
13.0        72
17.0        70
22.0        62
11.0        52
24.0        43
26.0        37
23.0        35
19.0        12
Name: MENTHLTH, dtype: int64

In [15]:
## _MICHD - check,replace,remove
 # change order, scale
  # 1 >> 1 (Reported having MI or CHD)
  # 2 >> 0 (Did not report having MI or CHD)
df_sel['_MICHD'].replace({1:1,2:0},inplace=True)
df_sel['_MICHD'] = df_sel['_MICHD'].astype(int)
df_sel['_MICHD'].value_counts()

0    223657
1     19277
Name: _MICHD, dtype: int64

In [16]:
## _RFCHOL1 - check,replace,remove
 # change order, scale
  # 1 >> 0 (No)
  # 2 >> 1 (Yes)
df_sel['_RFCHOL1'].replace({1:0,2:1},inplace=True)  
df_sel['_RFCHOL1'] = df_sel['_RFCHOL1'].astype(int)
df_sel['_RFCHOL1'].value_counts()

0    151570
1     91364
Name: _RFCHOL1, dtype: int64

In [17]:
## _RFHYPE5 - check,replace,remove
 # removing 9(don't know)
 # change order, scale
  # 1 >> 0 (No)
  # 2 >> 1 (Yes)
df_sel=df_sel[df_sel['_RFHYPE5'] != 9]
df_sel['_RFHYPE5'].replace({1:0,2:1},inplace=True) 
df_sel['_RFHYPE5'] = df_sel['_RFHYPE5'].astype(int)
df_sel['_RFHYPE5'].value_counts()

0    147714
1     94886
Name: _RFHYPE5, dtype: int64

In [18]:
## PA1VIGM_ - check,replace,remove
df_sel['PA1VIGM_'] = df_sel['PA1VIGM_'].round(0)
df_sel['PA1VIGM_'].value_counts()

0.0       127095
120.0       7045
180.0       6734
60.0        6577
90.0        5296
           ...  
3115.0         1
502.0          1
1321.0         1
879.0          1
2920.0         1
Name: PA1VIGM_, Length: 1536, dtype: int64

In [19]:
## _RFDRHV5 - check,replace,remove
  # 1 >> 0 (Not a heavy Drinker)
  # 2 >> 1 (Is Heavy Drinker)
  # Remove 9 >> (Don't know)
df_sel = df_sel[df_sel['_RFDRHV5'] != 9]
df_sel['_RFDRHV5'].replace({1:0,2:1},inplace=True)
df_sel['_RFDRHV5'] = df_sel['_RFDRHV5'].astype(int)
df_sel['_RFDRHV5'].value_counts()

0    224968
1     14506
Name: _RFDRHV5, dtype: int64

In [20]:
##  _RFSMOK3 - check,replace,remove
 # removing 9(don't know)
 # change order, scale
  # 1 >> 0 (No)
  # 2 >> 1 (Yes)
df_sel=df_sel[df_sel['_RFSMOK3'] != 9]
df_sel['_RFSMOK3'].replace({1:0,2:1},inplace=True) 
df_sel['_RFSMOK3'] = df_sel['_RFSMOK3'].astype(int)
df_sel['_RFSMOK3'].value_counts()

0    210694
1     27814
Name: _RFSMOK3, dtype: int64

In [21]:
##  _INCOMG - check,replace,remove
 # removing 9(don't know)
 # change order, scale
 # 1,2,3,4 ->> 0 (<$50,000)
 # 5 ->> 1 (>=$50,000)
df_sel=df_sel[df_sel['_INCOMG'] != 9]
df_sel['_INCOMG'].replace({1:0,2:0,3:0,4:0,5:1},inplace=True) 
df_sel['_INCOMG'] = df_sel['_INCOMG'].astype(int)
df_sel['_INCOMG'].value_counts()

1    120770
0     90896
Name: _INCOMG, dtype: int64

In [22]:
##  _EDUCAG - check,replace,remove
 # removing 9(don't know)
 # 1,2,3 ->> 0 (have not graduated college or technical school)
 # 4 ->> 1 (have graduated college or technical school)
df_sel=df_sel[df_sel['_EDUCAG'] != 9]
df_sel['_EDUCAG'].replace({1:0,2:0,3:0,4:1},inplace=True) 
df_sel['_EDUCAG'] = df_sel['_EDUCAG'].astype(int)
df_sel['_EDUCAG'].value_counts()

0    112150
1     99360
Name: _EDUCAG, dtype: int64

In [23]:
## SEX - check,replace,remove
  #  1 ->> 0 male
  #  2 ->> 1 female
  #  9 ->> refused (remove)
df_sel=df_sel[df_sel['SEX'] != 9]
df_sel['SEX'] = df_sel['SEX'].astype(int)
df_sel['SEX'].replace({1:0,2:1},inplace=True) 
df_sel['SEX'].value_counts()

1    112031
0     99479
Name: SEX, dtype: int64

In [24]:
## _AGE65YR - check,replace,remove
  #  1 ->> 0 18-64
  #  2 ->> 1 >=65
  #  3 ->> unknown (remove)
df_sel=df_sel[df_sel['_AGE65YR'] != 3]
df_sel['_AGE65YR'] = df_sel['_AGE65YR'].astype(int)
df_sel['_AGE65YR'].replace({1:0,2:1},inplace=True) 
df_sel['_AGE65YR'].value_counts()

0    139409
1     72101
Name: _AGE65YR, dtype: int64

In [25]:
df_sel['DIABETE3'].value_counts()

0    181714
1     29796
Name: DIABETE3, dtype: int64

In [26]:
df_sel.head()

,DIABETE3,_BMI5,PHYSHLTH,MENTHLTH,_MICHD,_RFCHOL1,_RFHYPE5,PA1VIGM_,_RFDRHV5,_RFSMOK3,_INCOMG,_EDUCAG,SEX,_AGE65YR
0,1,26.96,0.0,0.0,0,1,1,180.0,0,0,0,1,1,1
1,0,29.43,0.0,0.0,0,0,1,0.0,0,0,1,1,0,1
5,0,27.34,6.0,0.0,1,0,1,60.0,0,1,0,0,0,1
9,0,27.12,2.0,1.0,0,0,1,0.0,0,0,0,0,0,0
12,1,23.03,5.0,0.0,0,1,1,125.0,0,0,0,0,1,1


In [27]:
df_sel.columns = ['diabete', 'bmi', 'physhlth', 'menthlth', 'michd', 'hchol', 'hblpr', 'vpa'
                  , 'hvdr', 'smok', 'incom50', 'cllgr','sex', 'age65']

In [28]:
## save to CSV
df_sel.to_csv('../data/2017_diabetes_cleaned.csv', index = False)